In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import yfinance as yf
import time
import os

# Crypto categories to scrape from Yahoo Finance
category_urls = {
    "Crypto Most Active": "https://finance.yahoo.com/markets/crypto/most-active",
    "Crypto Gainers": "https://finance.yahoo.com/markets/crypto/gainers",
}

# Headers with contact email
headers = {
    "User-Agent": (
        "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
        "AppleWebKit/537.36 (KHTML, like Gecko) "
        "Chrome/122.0.0.0 Safari/537.36"
    ),
    "From": "hrahman@ucdavis.edu"
}

# Create required folders
os.makedirs("./historical-data/crypto", exist_ok=True)
os.makedirs("./market-data", exist_ok=True)

# Scrape up to 100 symbols from Yahoo table
def get_symbols_paginated(url, pages=1):
    results = []
    for page in range(pages):
        full_url = f"{url}?start={page * 100}&count=100"
        response = requests.get(full_url, headers=headers)
        soup = BeautifulSoup(response.text, "html.parser")
        rows = soup.select("table tbody tr")
        for row in rows:
            cols = row.find_all("td")
            if len(cols) >= 2:
                symbol = cols[0].text.strip()
                name = cols[1].text.strip()
                results.append((symbol, name))
            if len(results) >= 100:
                break
        if len(results) >= 100:
            break
    return results

# Compute % change using yfinance
def get_change_pct(ticker, period):
    try:
        hist = yf.Ticker(ticker).history(period=period)
        if hist.empty or len(hist["Close"]) < 2:
            return None
        return round(((hist["Close"].iloc[-1] - hist["Close"].iloc[0]) / hist["Close"].iloc[0]) * 100, 2)
    except:
        return None

# Get all performance windows
def get_all_changes(symbol):
    return {
        "Symbol": symbol,
        "1W Change %": get_change_pct(symbol, "5d"),
        "1M Change %": get_change_pct(symbol, "1mo"),
        "3M Change %": get_change_pct(symbol, "3mo"),
        "6M Change %": get_change_pct(symbol, "6mo"),
        "1Y Change %": get_change_pct(symbol, "1y"),
        "5Y Change %": get_change_pct(symbol, "5y")
    }

# Save full historical series
def save_historical_data(symbol, period="2y"):
    try:
        df = yf.Ticker(symbol).history(period=period, interval="1d")
        if df.empty:
            print(f"-> No historical data for {symbol}")
            return
        df.to_csv(f"historical-data/crypto/{symbol}.csv")
        print(f"-> Saved historical data for {symbol}")
    except Exception as e:
        print(f"-> Error for {symbol}: {e}")

# Main execution loop
all_data = []

for category, url in category_urls.items():
    print(f"-----Scraping: {category}")
    symbol_name_pairs = get_symbols_paginated(url, pages=1)
    for symbol, name in symbol_name_pairs:
        print(f"→ {symbol} | {name}")
        data = get_all_changes(symbol)
        data["Name"] = name
        data["Category"] = category
        all_data.append(data)

        save_historical_data(symbol)
        time.sleep(1)

df = pd.DataFrame(all_data)
df.to_csv("market-data/crypto-market-data.csv", index=False)
print("-> Saved to market-data/crypto-market-data.csv")
display(df.head())

-----Scraping: Crypto Most Active
→ USDT-USD | Tether USDt USD
-> Saved historical data for USDT-USD
→ BTC-USD | Bitcoin USD
-> Saved historical data for BTC-USD
→ MPRO31258-USD | Max Property USD
-> Saved historical data for MPRO31258-USD
→ DAI-USD | Dai USD
-> Saved historical data for DAI-USD
→ ETH-USD | Ethereum USD
-> Saved historical data for ETH-USD
→ USDC-USD | USD Coin USD
-> Saved historical data for USDC-USD
→ ZKJ-USD | Polyhedra Network USD
-> Saved historical data for ZKJ-USD
→ FDUSD-USD | First Digital USD USD
-> Saved historical data for FDUSD-USD
→ SOL-USD | Solana USD
-> Saved historical data for SOL-USD
→ WBNB-USD | Wrapped BNB USD
-> Saved historical data for WBNB-USD
→ XRP-USD | XRP USD
-> Saved historical data for XRP-USD
→ BNB-USD | BNB USD
-> Saved historical data for BNB-USD
→ PEPE24478-USD | Pepe USD


/var/folders/33/tmt_dn957d99k_t2zvbqlw_40000gn/T/ipykernel_19180/471772940.py:54: RuntimeWarning: divide by zero encountered in scalar divide
  return round(((hist["Close"].iloc[-1] - hist["Close"].iloc[0]) / hist["Close"].iloc[0]) * 100, 2)


-> Saved historical data for PEPE24478-USD
→ DOGE-USD | Dogecoin USD
-> Saved historical data for DOGE-USD
→ NXPC-USD | NEXPACE USD
-> Saved historical data for NXPC-USD
→ SOL16116-USD | Wrapped Solana USD
-> Saved historical data for SOL16116-USD
→ SUI20947-USD | Sui USD
-> Saved historical data for SUI20947-USD
→ WETH-USD | WETH USD
-> Saved historical data for WETH-USD
→ B2-USD | BSquared Network USD
-> Saved historical data for B2-USD
→ VBTC-USD | Venus BTC USD
-> Saved historical data for VBTC-USD
→ ADA-USD | Cardano USD
-> Saved historical data for ADA-USD
→ KOGE-USD | 48 Club Token USD
-> Saved historical data for KOGE-USD
→ VBNB-USD | Venus BNB USD
-> Saved historical data for VBNB-USD
→ AAVE-USD | Aave USD
-> Saved historical data for AAVE-USD
→ TRUMP35336-USD | OFFICIAL TRUMP USD
-> Saved historical data for TRUMP35336-USD
→ T  TRUMP-OFFICIAL-USD | TRUMP OFFICIAL USD


$T  TRUMP-OFFICIAL-USD: possibly delisted; no price data found  (period=3mo) (Yahoo error = "No data found, symbol may be delisted")
$T  TRUMP-OFFICIAL-USD: possibly delisted; no price data found  (period=6mo) (Yahoo error = "No data found, symbol may be delisted")
$T  TRUMP-OFFICIAL-USD: possibly delisted; no price data found  (period=1y) (Yahoo error = "No data found, symbol may be delisted")
$T  TRUMP-OFFICIAL-USD: possibly delisted; no price data found  (period=5y) (Yahoo error = "No data found, symbol may be delisted")
$T  TRUMP-OFFICIAL-USD: possibly delisted; no price data found  (period=2y) (Yahoo error = "No data found, symbol may be delisted")


-> No historical data for T  TRUMP-OFFICIAL-USD
→ MERL-USD | Merlin Chain USD
-> Saved historical data for MERL-USD
→ USD136148-USD | World Liberty Financial USD USD
-> Saved historical data for USD136148-USD
→ TRX-USD | TRON USD
-> Saved historical data for TRX-USD
→ WIF-USD | dogwifhat USD
-> Saved historical data for WIF-USD
→ VIRTUAL-USD | Virtuals Protocol USD
-> Saved historical data for VIRTUAL-USD
→ USDTZ30628-USD | Tether USD Bridged ZED20 USD
-> Saved historical data for USDTZ30628-USD
→ B-USD | BUILDon USD
-> Saved historical data for B-USD
→ HYPE32196-USD | Hyperliquid USD
-> Saved historical data for HYPE32196-USD
→ UNI7083-USD | Uniswap USD
-> Saved historical data for UNI7083-USD
→ ETHZ-USD | Ethereum Bridged ZED20 USD
-> Saved historical data for ETHZ-USD
→ LTC-USD | Litecoin USD
-> Saved historical data for LTC-USD
→ AVAX-USD | Avalanche USD
-> Saved historical data for AVAX-USD
→ LINK-USD | Chainlink USD
-> Saved historical data for LINK-USD
→ WLD-USD | Worldcoin USD


...

$M  MAMO-USD: possibly delisted; no price data found  (period=5d) (Yahoo error = "No data found, symbol may be delisted")
$M  MAMO-USD: possibly delisted; no price data found  (period=1mo) (Yahoo error = "No data found, symbol may be delisted")
$M  MAMO-USD: possibly delisted; no price data found  (period=3mo) (Yahoo error = "No data found, symbol may be delisted")
$M  MAMO-USD: possibly delisted; no price data found  (period=6mo) (Yahoo error = "No data found, symbol may be delisted")
$M  MAMO-USD: possibly delisted; no price data found  (period=1y) (Yahoo error = "No data found, symbol may be delisted")
$M  MAMO-USD: possibly delisted; no price data found  (period=5y) (Yahoo error = "No data found, symbol may be delisted")
$M  MAMO-USD: possibly delisted; no price data found  (period=2y) (Yahoo error = "No data found, symbol may be delisted")


-> No historical data for M  MAMO-USD
→ BROCCOLI35761-USD | Broccoli (broccolibnb.org) USD
-> Saved historical data for BROCCOLI35761-USD
→ KERMIT-USD | Kermit USD
-> Saved historical data for KERMIT-USD
→ FROG31851-USD | Frog USD


/var/folders/33/tmt_dn957d99k_t2zvbqlw_40000gn/T/ipykernel_19180/471772940.py:54: RuntimeWarning: divide by zero encountered in scalar divide
  return round(((hist["Close"].iloc[-1] - hist["Close"].iloc[0]) / hist["Close"].iloc[0]) * 100, 2)
/var/folders/33/tmt_dn957d99k_t2zvbqlw_40000gn/T/ipykernel_19180/471772940.py:54: RuntimeWarning: divide by zero encountered in scalar divide
  return round(((hist["Close"].iloc[-1] - hist["Close"].iloc[0]) / hist["Close"].iloc[0]) * 100, 2)
/var/folders/33/tmt_dn957d99k_t2zvbqlw_40000gn/T/ipykernel_19180/471772940.py:54: RuntimeWarning: divide by zero encountered in scalar divide
  return round(((hist["Close"].iloc[-1] - hist["Close"].iloc[0]) / hist["Close"].iloc[0]) * 100, 2)


-> Saved historical data for FROG31851-USD
→ SYMP-USD | Sympson AI USD
-> Saved historical data for SYMP-USD
→ IMT36285-USD | Immortal Token USD
-> Saved historical data for IMT36285-USD
→ SQUID33447-USD | Squid Game (CTO) USD
-> Saved historical data for SQUID33447-USD
→ PHTR-USD | Phuture USD
-> Saved historical data for PHTR-USD
→ MMSS-USD | MMSS (Ordinals) USD
-> Saved historical data for MMSS-USD
→ GTC-USD | Game.com USD
-> Saved historical data for GTC-USD
→ NAIIVE-USD | Naiive USD
-> Saved historical data for NAIIVE-USD
→ PERRY35824-USD | Perry USD
-> Saved historical data for PERRY35824-USD
→ NOMOX-USD | NOMOEX Token USD
-> Saved historical data for NOMOX-USD
→ VERT-USD | Vertus USD
-> Saved historical data for VERT-USD
→ TRT27458-USD | TRUST AI USD
-> Saved historical data for TRT27458-USD
→ SAY-USD | SAY Coin USD
-> Saved historical data for SAY-USD
→ ADF24796-USD | Art de Finance USD
-> Saved historical data for ADF24796-USD
→ GORILLA36438-USD | Gorilla USD
-> Saved historic

,Symbol,1W Change %,1M Change %,3M Change %,6M Change %,1Y Change %,5Y Change %,Name,Category
0,USDT-USD,0.02,-0.01,0.13,-0.07,0.11,-0.42,Tether USDt USD,Crypto Most Active
1,BTC-USD,1.83,16.53,28.98,13.85,57.43,1089.96,Bitcoin USD,Crypto Most Active
2,MPRO31258-USD,-9.14,-53.89,-73.80,-99.32,-94.89,-95.37,Max Property USD,Crypto Most Active
3,DAI-USD,-0.01,-0.00,-0.01,0.00,0.01,-0.95,Dai USD,Crypto Most Active
4,ETH-USD,1.18,42.59,10.88,-30.10,-34.32,1123.95,Ethereum USD,Crypto Most Active
